# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 11 2022 3:03PM,243994,19,ADV80004822,"AdvaGen Pharma, Ltd",Released
1,Aug 11 2022 2:58PM,243991,10,8390129,Yusen – 3D Matrix,Released
2,Aug 11 2022 2:54PM,243990,10,0085935336,ISDIN Corporation,Released
3,Aug 11 2022 2:54PM,243990,10,0085935340,ISDIN Corporation,Released
4,Aug 11 2022 2:54PM,243990,10,0085935341,ISDIN Corporation,Released
5,Aug 11 2022 2:54PM,243990,10,0085935342,ISDIN Corporation,Released
6,Aug 11 2022 2:54PM,243990,10,0085935343,ISDIN Corporation,Released
7,Aug 11 2022 2:54PM,243990,10,0085935346,ISDIN Corporation,Released
8,Aug 11 2022 2:54PM,243990,10,0085935344,ISDIN Corporation,Released
9,Aug 11 2022 2:54PM,243990,10,0085935347,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,243988,Released,1
27,243989,Released,63
28,243990,Released,41
29,243991,Released,1
30,243994,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
243988,NaN,NaN,1.0
243989,NaN,NaN,63.0
243990,NaN,NaN,41.0
243991,NaN,NaN,1.0
243994,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
243912,0.0,0.0,3.0
243913,0.0,0.0,5.0
243934,0.0,0.0,32.0
243942,0.0,0.0,43.0
243943,0.0,0.0,16.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
243912,0.0,0,3
243913,0.0,0,5
243934,0.0,0,32
243942,0.0,0,43
243943,0.0,0,16


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,243912,0.0,0,3
1,243913,0.0,0,5
2,243934,0.0,0,32
3,243942,0.0,0,43
4,243943,0.0,0,16


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,243912,0.0,,3
1,243913,0.0,,5
2,243934,0.0,,32
3,243942,0.0,,43
4,243943,0.0,,16


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 11 2022 3:03PM,243994,19,"AdvaGen Pharma, Ltd"
1,Aug 11 2022 2:58PM,243991,10,Yusen – 3D Matrix
2,Aug 11 2022 2:54PM,243990,10,ISDIN Corporation
43,Aug 11 2022 2:54PM,243989,10,ISDIN Corporation
106,Aug 11 2022 2:47PM,243988,10,Eywa Pharma Inc.
107,Aug 11 2022 2:44PM,243983,10,Bio-PRF
111,Aug 11 2022 2:38PM,243987,10,Emerginnova
112,Aug 11 2022 1:51PM,243985,19,ACG North America LLC
113,Aug 11 2022 1:44PM,243984,10,ISDIN Corporation
114,Aug 11 2022 12:59PM,243975,10,Beach Products Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,Aug 11 2022 3:03PM,243994,19,"AdvaGen Pharma, Ltd",0.0,,1
1,Aug 11 2022 2:58PM,243991,10,Yusen – 3D Matrix,0.0,,1
2,Aug 11 2022 2:54PM,243990,10,ISDIN Corporation,0.0,,41
3,Aug 11 2022 2:54PM,243989,10,ISDIN Corporation,0.0,,63
4,Aug 11 2022 2:47PM,243988,10,Eywa Pharma Inc.,0.0,,1
5,Aug 11 2022 2:44PM,243983,10,Bio-PRF,0.0,,4
6,Aug 11 2022 2:38PM,243987,10,Emerginnova,0.0,,1
7,Aug 11 2022 1:51PM,243985,19,ACG North America LLC,0.0,,1
8,Aug 11 2022 1:44PM,243984,10,ISDIN Corporation,0.0,,1
9,Aug 11 2022 12:59PM,243975,10,Beach Products Inc,0.0,13,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 11 2022 3:03PM,243994,19,"AdvaGen Pharma, Ltd",1,
1,Aug 11 2022 2:58PM,243991,10,Yusen – 3D Matrix,1,
2,Aug 11 2022 2:54PM,243990,10,ISDIN Corporation,41,
3,Aug 11 2022 2:54PM,243989,10,ISDIN Corporation,63,
4,Aug 11 2022 2:47PM,243988,10,Eywa Pharma Inc.,1,
5,Aug 11 2022 2:44PM,243983,10,Bio-PRF,4,
6,Aug 11 2022 2:38PM,243987,10,Emerginnova,1,
7,Aug 11 2022 1:51PM,243985,19,ACG North America LLC,1,
8,Aug 11 2022 1:44PM,243984,10,ISDIN Corporation,1,
9,Aug 11 2022 12:59PM,243975,10,Beach Products Inc,4,13


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 11 2022 3:03PM,243994,19,"AdvaGen Pharma, Ltd",1,
1,Aug 11 2022 2:58PM,243991,10,Yusen – 3D Matrix,1,
2,Aug 11 2022 2:54PM,243990,10,ISDIN Corporation,41,
3,Aug 11 2022 2:54PM,243989,10,ISDIN Corporation,63,
4,Aug 11 2022 2:47PM,243988,10,Eywa Pharma Inc.,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 11 2022 3:03PM,243994,19,"AdvaGen Pharma, Ltd",1.0,NaN
1,Aug 11 2022 2:58PM,243991,10,Yusen – 3D Matrix,1.0,NaN
2,Aug 11 2022 2:54PM,243990,10,ISDIN Corporation,41.0,NaN
3,Aug 11 2022 2:54PM,243989,10,ISDIN Corporation,63.0,NaN
4,Aug 11 2022 2:47PM,243988,10,Eywa Pharma Inc.,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 11 2022 3:03PM,243994,19,"AdvaGen Pharma, Ltd",1.0,0.0
1,Aug 11 2022 2:58PM,243991,10,Yusen – 3D Matrix,1.0,0.0
2,Aug 11 2022 2:54PM,243990,10,ISDIN Corporation,41.0,0.0
3,Aug 11 2022 2:54PM,243989,10,ISDIN Corporation,63.0,0.0
4,Aug 11 2022 2:47PM,243988,10,Eywa Pharma Inc.,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4147533,222.0,20.0
15,1219649,12.0,3.0
17,731739,15.0,0.0
19,487979,2.0,0.0
20,487924,31.0,5.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4147533,222.0,20.0
1,15,1219649,12.0,3.0
2,17,731739,15.0,0.0
3,19,487979,2.0,0.0
4,20,487924,31.0,5.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,222.0,20.0
1,15,12.0,3.0
2,17,15.0,0.0
3,19,2.0,0.0
4,20,31.0,5.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,222.0
1,15,Released,12.0
2,17,Released,15.0
3,19,Released,2.0
4,20,Released,31.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,17,19,20
Status,,,,,
Executing,20.0,3.0,0.0,0.0,5.0
Released,222.0,12.0,15.0,2.0,31.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,17,19,20
0,Executing,20.0,3.0,0.0,0.0,5.0
1,Released,222.0,12.0,15.0,2.0,31.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,17,19,20
0,Executing,20.0,3.0,0.0,0.0,5.0
1,Released,222.0,12.0,15.0,2.0,31.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()